## Paquets i dades originals ## 

In [ ]:
import json
import pandas as pd
from datetime import datetime
from listas import disfagia_list, PA_list, P_list, Main_respiratory_infections_list, LRTI_list, COPD_exacerbations_list, Pulmonary_fibrosis_fibrotorax_list
from auxiliary_functions import valores_codigos, contar_diccionarios, dias_ingreso_total, asignar_intervalo_edad, sumar_barthel, sumar_emina, obtener_ultimo_resultat, obtener_valor_promedio, canadenca_comparada, disfagia_mecvvs, extraer_valor_clave, extraer_valor_clave_simple

tabla = pd.read_pickle('./data/processed/dataframe.pkl')
import scipy.stats as stats
from scipy.stats import ttest_ind
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu
from scipy.stats import chi2_contingency


Prèviament, he tractat les dades creant noves columnes que classifiquen millor tota la informació disponible

In [ ]:
tabla.head()

Distribuiré les dades en 3 grups:
- **CON_PA**: aquest grup està compost per pacients que contenen el codi ICD de pneumònia aspirativa al seu historial.
- **CON_PA_MECVV**: aquest grup conté els pacients que compleixen les següents característiques: 
    - Han donat positiu al MECVV (és a dir, que tenen disfàgia i una alteració en la seguretat o en l'eficàcia)
    - Contenen el codi ICD de pneumònia
    - Han estat ingressats menys de 30 dies en total. 
- **SIN_PA**: aquest grup està format per pacients que contenen el codi ICD de pneumònia o bé:
    - Contenen el codi ICD de pneumònia
    - Han donat positiu al MECVV 
    - Han estat ingressats més de 30 dies en total.

# CON_PA 

In [ ]:
CON_PA = tabla[tabla["PA_diagnosticada"] == 1.0]
CON_PA.head(len(tabla))

# CON_PA_MECVV

In [ ]:
CON_PA_MECVV = tabla[
    (tabla["P_diagnosticada"] == 1.0) & (tabla["Dias_totales_ingresado"] < 30) & (tabla["Disfagia_mecvvs"] == 1.0) & (tabla["alteracioEficacia_mecvvs"] == 1.0) |
    (tabla["P_diagnosticada"] == 1.0) & (tabla["Dias_totales_ingresado"] < 30) & (tabla["Disfagia_mecvvs"] == 1.0) & (tabla["alteracioSeguretat_mecvvs"] == 1.0)]   
CON_PA_MECVV.head(len(tabla))

# SIN_PA

In [ ]:
SIN_PA = tabla[
          (tabla["P_diagnosticada"] == 1.0) |
          (tabla["P_diagnosticada"] == 1.0) & (tabla["Disfagia_mecvvs"] == 1.0) & (tabla["alteracioEficacia_mecvvs"] == 1.0) & (tabla["Dias_totales_ingresado"] > 30) |
          (tabla["P_diagnosticada"] == 1.0) & (tabla["Disfagia_mecvvs"] == 1.0) & (tabla["alteracioSeguretat_mecvvs"] == 1.0) & (tabla["Dias_totales_ingresado"] > 30)]
SIN_PA.head(len(tabla))

Ara que tinc totes les dades classificades en aquests 3 grups, començaré a analitzar el seu contingut utilitzant operacions com mitjanes i desviacions estàndard. També compararé els diferents grups amb el pvalor mitjançant T-test, test de Mann-Whitney o test Xi-quadrat. Els criteris que seguiré per utilitzar un test o un altre és fer un test de Shapiro-Wilk per comprovar si segueixen una distribució normal, si és així, usaré el T-test, si no, utilitzaré el de Mann-Whitney. Finalment, si la columna conté valors categòrics com és el cas del sexe (F/M), faré servir el test Xi-quadrat.


## Número

In [ ]:
num_CON_PA = len(CON_PA)
num_CON_PA_MECVV= len(CON_PA_MECVV)
num_SIN_PA = len(SIN_PA)

print(num_CON_PA, num_CON_PA_MECVV, num_SIN_PA)

Això també es pot comprovar en observar les files que conté les taules anteriors de cada grup, on veiem que la taula de CON_PA té 17 files, la de CON_PA_MECV 10 i la de SIN_PA 49. 

#### Pvalor

##### Normalitat

In [ ]:
stat, p_value = shapiro(range(num_CON_PA))
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
stat, p_value = shapiro(range(num_CON_PA_MECVV))
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
stat, p_value = shapiro(range(num_SIN_PA))
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
t_stat_12, p_val_12 = ttest_ind(CON_PA.index, CON_PA_MECVV.index, equal_var=False)
print(f"P-valor entre CON_PA i CON_PA_MECVV: {p_val_12:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

In [1495]:
t_stat_13, p_val_13 = ttest_ind(CON_PA.index, SIN_PA.index, equal_var=False)
print(f"P-valor entre CON_PA i SIN_PA: {p_val_13:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

P-valor entre CON_PA i SIN_PA: 0.7664
No existeixen diferències significatives entre els 2 grups.


In [1496]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV.index, SIN_PA.index, equal_var=False)
print(f"P-valor entre CON_PA_MECVV i SIN_PA: {p_val_12:.4f}")
alpha = 0.05
if p_value < alpha:
    print("Existeixen diferències significatives entre els 2 grups.")
else:
    print("No existeixen diferències significatives entre els 2 grups.")

P-valor entre CON_PA_MECVV i SIN_PA: nan
No existeixen diferències significatives entre els 2 grups.


## Edat

In [1497]:
edad_CON_PA = CON_PA['edat'].mean()
desviacion_estandar_edad = CON_PA['edat'].std()

resultado = f"{edad_CON_PA:.2f} ± {desviacion_estandar_edad:.2f}"
print("Mitjana ± Desviació Estàndard de l'edat:", resultado)


Mitjana ± Desviació Estàndard de l'edat: 87.06 ± 8.99


In [1498]:
edad_CON_PA_MECVV = CON_PA_MECVV['edat'].mean()
desviacion_estandar_edad = CON_PA_MECVV['edat'].std()

resultado = f"{edad_CON_PA_MECVV:.2f} ± {desviacion_estandar_edad:.2f}"
print("Mitjana ± Desviació Estàndard de l'edat:", resultado)

Mitjana ± Desviació Estàndard de l'edat: 88.90 ± 8.67


In [1499]:
edad_SIN_PA = SIN_PA['edat'].mean()
desviacion_estandar_edad = SIN_PA['edat'].std()

resultado = f"{edad_SIN_PA:.2f} ± {desviacion_estandar_edad:.2f}"
print("Mitjana ± Desviació Estàndard de l'edat:", resultado)

Mitjana ± Desviació Estàndard de l'edat: 80.12 ± 17.01


#### Pvalor

##### Normalitat 

In [1500]:
stat, p_value = shapiro(CON_PA['edat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9410
Valor p = 0.3297
El número de files segueix una distribució normal.


In [1501]:
stat, p_value = shapiro(CON_PA_MECVV['edat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9057
Valor p = 0.2528
El número de files segueix una distribució normal.


In [1502]:
stat, p_value = shapiro(SIN_PA['edat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8539
Valor p = 0.0000
El número de files no segueix una distribució normal.


In [1503]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['edat'], CON_PA_MECVV['edat'], equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: 0.6051223082851969


In [1504]:
stat, p_value_13 = mannwhitneyu(CON_PA['edat'], SIN_PA['edat'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA i SIN_PA 0.1815160884632735


In [1505]:
stat_23, p_value_23 = mannwhitneyu(CON_PA_MECVV['edat'], SIN_PA['edat'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_value_23)

P-valor entre CON_PA_MECVV i SIN_PA: 0.11458529353557328


## Sexe

In [1506]:
sexe_CON_PA = CON_PA['sexe'].value_counts()
sexe_CON_PA_MECVV = CON_PA_MECVV['sexe'].value_counts()
sexe_SIN_PA = SIN_PA['sexe'].value_counts()

In [1507]:
if 'F' in sexe_CON_PA:
    percent_female1 = (sexe_CON_PA['F'] / len(CON_PA)) * 100
    print(f"Percentatge de sexe femení en CON_PA: {percent_female1:.2f}%")
else:
    print("No hi ha dades de sexe femení en CON_PA")

Percentatge de sexe femení en CON_PA: 47.06%


In [1508]:
if 'F' in sexe_CON_PA_MECVV:
    percent_female2 = (sexe_CON_PA_MECVV['F'] / len(CON_PA)) * 100
    print(f"Percentatge de sexe femení en CON_PA_MECVV: {percent_female1:.2f}%")
else:
    print("No hi ha dades de sexe femení en CON_PA_MECVV")

Percentatge de sexe femení en CON_PA_MECVV: 47.06%


In [1509]:
if 'F' in sexe_SIN_PA:
    percent_female3 = (sexe_SIN_PA['F'] / len(SIN_PA)) * 100
    print(f"Percentatge de sexe femení en SIN_PA: {percent_female1:.2f}%")
else:
    print("No hi ha dades de sexe femení en SIN_PA")


Percentatge de sexe femení en SIN_PA: 47.06%


#### Pvalor

##### Normalitat

Al ser una variable categòrica no es pot comprovar la seva normalitat. Realitzaré directament el test xi-quadrat

In [1510]:
contingency_table_CON_PA_CON_PA_MECVV = pd.crosstab(index=CON_PA['sexe'], columns=CON_PA_MECVV['sexe'])

chi2_CON_PA_CON_PA_MECVV, p_value_CON_PA_CON_PA_MECVV, dof_CON_PA_CON_PA_MECVV, expected_CON_PA_CON_PA_MECVV = chi2_contingency(contingency_table_CON_PA_CON_PA_MECVV)

print("Estadístic xi-quadrat entre CON_PA i CON_PA_MECVV:", chi2_CON_PA_CON_PA_MECVV)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_value_CON_PA_CON_PA_MECVV)

if p_value_CON_PA_CON_PA_MECVV < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en CON_PA i CON_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en CON_PA i CON_PA_MECVV.")

Estadístic xi-quadrat entre CON_PA i CON_PA_MECVV: 2.6666666666666665
P-valor entre CON_PA i CON_PA_MECVV: 0.10247043485974942
No hi ha diferències significatives entre les distribucions de sexe en CON_PA i CON_PA_MECVV.


In [1511]:
contingency_table_CON_PA_SIN_PA = pd.crosstab(index=CON_PA['sexe'], columns=SIN_PA['sexe'])

chi2_CON_PA_SIN_PA, p_value_CON_PA_SIN_PA, dof_CON_PA_SIN_PA, expected_CON_PA_SIN_PA = chi2_contingency(contingency_table_CON_PA_SIN_PA)

print("Estadístic xi-quadrat entre CON_PA i SIN_PA:", chi2_CON_PA_SIN_PA)
print("P-valor entre CON_PA i SIN_PA:", p_value_CON_PA_SIN_PA)

if p_value_CON_PA_SIN_PA < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en CON_PA i SIN_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en CON_PA i SIN_PA.")

Estadístic xi-quadrat entre CON_PA i SIN_PA: 13.223042052469136
P-valor entre CON_PA i SIN_PA: 0.00027652843917458906
Hi ha diferències significatives entre les distribucions de sexe en CON_PA i SIN_PA.


In [1512]:
contingency_table_CON_PA_MECVV_SIN_PA = pd.crosstab(index=CON_PA_MECVV['sexe'], columns=SIN_PA['sexe'])

chi2_CON_PA_MECVV_SIN_PA, p_value_CON_PA_MECVV_SIN_PA, dof_CON_PA_MECVV_SIN_PA, expected_CON_PA_MECVV_SIN_PA = chi2_contingency(contingency_table_CON_PA_MECVV_SIN_PA)

print("Estadístic xi-quadrat entre CON_PA_MECVV i SIN_PA:", chi2_CON_PA_MECVV_SIN_PA)
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_value_CON_PA_MECVV_SIN_PA)

if p_value_CON_PA_MECVV_SIN_PA < 0.05:
    print("Hi ha diferències significatives entre les distribucions de sexe en CON_PA_MECVV i SIN_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de sexe en CON_PA_MECVV i SIN_PA.")

Estadístic xi-quadrat entre CON_PA_MECVV i SIN_PA: 6.267361111111111
P-valor entre CON_PA_MECVV i SIN_PA: 0.012298217061234292
Hi ha diferències significatives entre les distribucions de sexe en CON_PA_MECVV i SIN_PA.


## Ingressos (admissions)

In [1513]:
promedio_ingresos = CON_PA['Num_ingresos'].mean()
desviacion_estandar_ingresos = CON_PA['Num_ingresos'].std()

resultado = f"{promedio_ingresos:.2f} ± {desviacion_estandar_ingresos:.2f}"
print("Mitjana ± Desviació Estàndard del nombre d'ingressos:", resultado)

Mitjana ± Desviació Estàndard del nombre d'ingressos: 5.59 ± 3.37


In [1514]:
promedio_ingresos = CON_PA_MECVV['Num_ingresos'].mean()
desviacion_estandar_ingresos = CON_PA_MECVV['Num_ingresos'].std()

resultado = f"{promedio_ingresos:.2f} ± {desviacion_estandar_ingresos:.2f}"
print("Mitjana ± Desviació Estàndard del nombre d'ingressos:", resultado)

Mitjana ± Desviació Estàndard del nombre d'ingressos: 5.90 ± 2.08


In [1515]:
promedio_ingresos = SIN_PA['Num_ingresos'].mean()
desviacion_estandar_ingresos = SIN_PA['Num_ingresos'].std()

resultado = f"{promedio_ingresos:.2f} ± {desviacion_estandar_ingresos:.2f}"
print("Mitjana ± Desviació Estàndard del nombre d'ingressos:", resultado)

Mitjana ± Desviació Estàndard del nombre d'ingressos: 6.69 ± 5.13


#### Pvalor

##### Normalitat

In [1516]:
stat, p_value = shapiro(CON_PA['Num_ingresos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8008
Valor p = 0.0021
El número de files no segueix una distribució normal.


In [1517]:
stat, p_value = shapiro(CON_PA_MECVV['Num_ingresos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9558
Valor p = 0.7376
El número de files segueix una distribució normal.


In [1518]:
stat, p_value = shapiro(SIN_PA['Num_ingresos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.7277
Valor p = 0.0000
El número de files no segueix una distribució normal.


In [1519]:
stat, p_value_12 = mannwhitneyu(CON_PA['Num_ingresos'], CON_PA_MECVV['Num_ingresos'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV 0.3599581940859352


In [1520]:
stat, p_value_13 = mannwhitneyu(CON_PA['Num_ingresos'], SIN_PA['Num_ingresos'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA i SIN_PA 0.514931142191354


In [1521]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['Num_ingresos'], SIN_PA['Num_ingresos'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV i SIN_PA 0.7831397976578486


## Disfàgia

In [1522]:
disf_CON_PA= (CON_PA['DO_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de disfàgia: {disf_CON_PA:.2f}%")

Percentatge de disfàgia: 47.06%


In [1523]:
disf_CON_PA_MECVV = (CON_PA_MECVV['DO_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de disfàgia: {disf_CON_PA_MECVV:.2f}%")

Percentatge de disfàgia: 30.00%


In [1524]:
disf_SIN_PA= (SIN_PA['DO_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de disfàgia: {disf_SIN_PA:.2f}%")

Percentatge de disfàgia: 18.37%


#### Pvalor

##### Normalitat

In [1525]:
stat, p_value = shapiro(CON_PA['DO_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.6418
Valor p = 0.0000
El número de files no segueix una distribució normal.


In [1526]:
stat, p_value = shapiro(CON_PA_MECVV['DO_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.5942
Valor p = 0.0000
El número de files no segueix una distribució normal.


In [1527]:
stat, p_value = shapiro(SIN_PA['DO_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.4716
Valor p = 0.0000
El número de files no segueix una distribució normal.


In [1528]:
stat, p_value_12 = mannwhitneyu(CON_PA['DO_diagnosticada'], CON_PA_MECVV['DO_diagnosticada'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV 0.40914513229166993


In [1529]:
stat, p_value_13 = mannwhitneyu(CON_PA['DO_diagnosticada'], SIN_PA['DO_diagnosticada'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA i SIN_PA 0.02125060274444335


In [1530]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['DO_diagnosticada'], SIN_PA['DO_diagnosticada'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV i SIN_PA 0.4172116903428992


## Index de Barthel

### Promedio

In [1531]:
promedio_barthel = CON_PA['Barthel_resultados'].mean()
desviacion_estandar_barthel = CON_PA['Barthel_resultados'].std()

resultado = f"{promedio_barthel:.2f} ± {desviacion_estandar_barthel:.2f}"
print("Mitjana ± Desviació Estàndard del Barthel:", resultado)

Mitjana ± Desviació Estàndard del Barthel: 22.65 ± 24.63


In [1532]:
promedio_barthel = CON_PA_MECVV['Barthel_resultados'].mean()
desviacion_estandar_barthel = CON_PA_MECVV['Barthel_resultados'].std()

resultado = f"{promedio_barthel:.2f} ± {desviacion_estandar_barthel:.2f}"
print("Mitjana ± Desviació Estàndard del Barthel:", resultado)

Mitjana ± Desviació Estàndard del Barthel: 28.50 ± 22.24


In [1533]:
promedio_barthel = SIN_PA['Barthel_resultados'].mean()
desviacion_estandar_barthel = SIN_PA['Barthel_resultados'].std()

resultado = f"{promedio_barthel:.2f} ± {desviacion_estandar_barthel:.2f}"
print("Mitjana ± Desviació Estàndard del Barthel:", resultado)

Mitjana ± Desviació Estàndard del Barthel: 48.88 ± 37.42


#### Pvalor

##### Normalitat

In [1534]:
stat, p_value = shapiro(CON_PA['Barthel_resultados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8517
Valor p = 0.0114
El número de files no segueix una distribució normal.


In [1535]:
stat, p_value = shapiro(CON_PA_MECVV['Barthel_resultados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9138
Valor p = 0.3083
El número de files segueix una distribució normal.


In [1536]:
stat, p_value = shapiro(SIN_PA['Barthel_resultados'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8858
Valor p = 0.0002
El número de files no segueix una distribució normal.


In [1537]:
stat, p_value_12 = mannwhitneyu(CON_PA['Barthel_resultados'], CON_PA_MECVV['Barthel_resultados'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV", p_value_12)

P-valor entre CON_PA i CON_PA_MECVV 0.41631085084208264


In [1538]:
stat, p_value_13 = mannwhitneyu(CON_PA['Barthel_resultados'], SIN_PA['Barthel_resultados'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA", p_value_13)

P-valor entre CON_PA i SIN_PA 0.012373091641540783


In [1539]:
stat, p_value_23 = mannwhitneyu(CON_PA_MECVV['Barthel_resultados'], SIN_PA['Barthel_resultados'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA", p_value_23)

P-valor entre CON_PA_MECVV i SIN_PA 0.13803563369050034


### Valors de dependència

##### CON_PA

In [1540]:
indepe_CON_PA = (CON_PA['Barthel_resultados'] == 100.0).mean() * 100
print(f"Percentatge de indepe_CON_PA: {indepe_CON_PA:.2f}%")

Percentatge de indepe_CON_PA: 0.00%


In [1541]:
mod_CON_PA = (CON_PA['Barthel_resultados'].between(61, 95)).mean() * 100
print(f"Percentatge de mod_CON_PA: {mod_CON_PA:.2f}%")

Percentatge de mod_CON_PA: 5.88%


In [1542]:
sev_CON_PA = (CON_PA['Barthel_resultados'].between(21, 60)).mean() * 100
print(f"Percentatge de sev_CON_PA: {sev_CON_PA:.2f}%")

Percentatge de sev_CON_PA: 41.18%


In [1543]:
total_CON_PA = (CON_PA['Barthel_resultados'] <= 20.0 ).mean() * 100
print(f"Percentatge de total_CON_PA: {total_CON_PA:.2f}%")

Percentatge de total_CON_PA: 52.94%


##### CON_PA_MECVV

In [1544]:
indepe_CON_PA_MECVV = (CON_PA_MECVV['Barthel_resultados'] == 100.0).mean() * 100
print(f"Percentatge de indepe_CON_PA_MECVV: {indepe_CON_PA_MECVV:.2f}%")

Percentatge de indepe_CON_PA_MECVV: 0.00%


In [1545]:
mod_CON_PA_MECVV = (CON_PA_MECVV['Barthel_resultados'].between(61,95)).mean() * 100
print(f"Percentatge de mod_CON_PA_MECVV: {mod_CON_PA_MECVV:.2f}%")

Percentatge de mod_CON_PA_MECVV: 0.00%


In [1546]:
sev_CON_PA_MECVV = (CON_PA_MECVV['Barthel_resultados'].between(21,60)).mean() * 100
print(f"Percentatge de sev_CON_PA_MECVV: {sev_CON_PA_MECVV:.2f}%")

Percentatge de sev_CON_PA_MECVV: 60.00%


In [1547]:
total_CON_PA_MECVV = (CON_PA_MECVV['Barthel_resultados'] <= 20.0).mean() * 100
print(f"Percentatge de total_CON_PA_MECVV: {total_CON_PA_MECVV:.2f}%")

Percentatge de total_CON_PA_MECVV: 40.00%


##### SIN_PA

In [1548]:
indepe_SIN_PA = (SIN_PA['Barthel_resultados'] == 100.0).mean() * 100
print(f"Percentatge de indepe_SIN_PA: {indepe_SIN_PA:.2f}%")

Percentatge de indepe_SIN_PA: 20.41%


In [1549]:
mod_SIN_PA = (SIN_PA['Barthel_resultados'].between(61,95)).mean() * 100
print(f"Percentatge de mod_SIN_PA: {mod_SIN_PA:.2f}%")

Percentatge de mod_SIN_PA: 18.37%


In [1550]:
sev_SIN_PA = (SIN_PA['Barthel_resultados'].between(21,60)).mean() * 100
print(f"Percentatge de sev_SIN_PA: {sev_SIN_PA:.2f}%")

Percentatge de sev_SIN_PA: 28.57%


In [1551]:
total_SIN_PA = (SIN_PA['Barthel_resultados'] <= 20).mean() * 100
print(f"Percentatge de total_SIN_PA: {total_SIN_PA:.2f}%")

Percentatge de total_SIN_PA: 32.65%


#### Pvalor

##### Independència

In [1552]:
stat_12, p_value_12 = mannwhitneyu(CON_PA['Barthel_resultados'] == 100.0, CON_PA_MECVV['Barthel_resultados'] == 100.0, alternative='two-sided')
print("P-valor entre indepe_CON_PA i indepe_CON_PA_MECVV:", p_value_12)

P-valor entre indepe_CON_PA i indepe_CON_PA_MECVV: 1.0


In [1553]:
stat_13, p_value_13 = mannwhitneyu(CON_PA['Barthel_resultados'] == 100.0, SIN_PA['Barthel_resultados'] == 100.0, alternative='two-sided')
print("P-valor entre indepe_CON_PA i indepe_SIN_PA:", p_value_13)

P-valor entre indepe_CON_PA i indepe_SIN_PA: 0.04605091215969595


In [1554]:
stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['Barthel_resultados'] == 100.0, SIN_PA['Barthel_resultados'] == 100.0, equal_var=False)
print("P-valor entre indepe_CON_PA_MECVV i SIN_PA:", p_value_23)

P-valor entre indepe_CON_PA_MECVV i SIN_PA: 0.13803563369050034


##### Moderada

In [1555]:
stat_12, p_value_12 = mannwhitneyu(CON_PA['Barthel_resultados'].between(61, 95), CON_PA_MECVV['Barthel_resultados'].between(61, 95), alternative='two-sided')
print("P-valor entre mod_CON_PA i mod_CON_PA_MECVV:", p_value_12)

P-valor entre mod_CON_PA i mod_CON_PA_MECVV: 0.49002535950179893


In [1556]:
stat_13, p_value_13 = mannwhitneyu(CON_PA['Barthel_resultados'].between(61, 95), SIN_PA['Barthel_resultados'].between(61, 95), alternative='two-sided')
print("P-valor entre indepe_CON_PA i mod_SIN_PA:", p_value_13)

P-valor entre indepe_CON_PA i mod_SIN_PA: 0.22404485220846948


In [1557]:
stat_23, p_value_23 = mannwhitneyu(CON_PA_MECVV['Barthel_resultados'].between(61, 95), SIN_PA['Barthel_resultados'].between(61, 95), alternative='two-sided')  
print("P-valor entre mod_CON_PA_MECVV i mod_SIN_PA :", p_value_23)

P-valor entre mod_CON_PA_MECVV i mod_SIN_PA : 0.14889565499123467


##### Severa

In [1558]:
stat_12, p_value_12 = mannwhitneyu(CON_PA['Barthel_resultados'].between(21,60), CON_PA_MECVV['Barthel_resultados'].between(21,60), alternative='two-sided')
print("P-valor entre sev_CON_PA i sev_CON_PA_MECVV:", p_value_12)

P-valor entre sev_CON_PA i sev_CON_PA_MECVV: 0.3688417252786701


In [1559]:
stat_13, p_value_13 = mannwhitneyu(CON_PA['Barthel_resultados'].between(21,60), SIN_PA['Barthel_resultados'].between(21,60), alternative='two-sided')
print("P-valor entre sev_CON_PA i sev_SIN_PA:", p_value_13)

P-valor entre sev_CON_PA i sev_SIN_PA: 0.3446368758082651


In [1560]:
t_stat_23, p_value_23 = mannwhitneyu(CON_PA_MECVV['Barthel_resultados'].between(21,60), SIN_PA['Barthel_resultados'].between(21,60), alternative='two-sided' )
print("P-valor entre sev_CON_PA_MECVV i sev_SIN_PA:", p_value_23)

P-valor entre sev_CON_PA_MECVV i sev_SIN_PA: 0.05945988532601827


##### Total

In [1561]:
stat_12, p_value_12 = mannwhitneyu(CON_PA['Barthel_resultados'] <= 20.0, CON_PA_MECVV['Barthel_resultados'] <= 20.0, alternative='two-sided')
print("P-valor entre total_CON_PA i total_CON_PA_MECVV:", p_value_12)

P-valor entre total_CON_PA i total_CON_PA_MECVV: 0.5426838040005246


In [1562]:
stat_13, p_value_13 = mannwhitneyu(CON_PA['Barthel_resultados'] <= 20.0, SIN_PA['Barthel_resultados'] <= 20.0, alternative='two-sided')
print("P-valor entre total_CON_PA i total_SIN_PA:", p_value_13)

P-valor entre total_CON_PA i total_SIN_PA: 0.14269651822055499


In [1563]:
stat_23, p_value_23 = mannwhitneyu(CON_PA_MECVV['Barthel_resultados'] <= 20.0, SIN_PA['Barthel_resultados'] <= 20.0, alternative='two-sided')
print("P-valor entre total_CON_PA_MECVV i total_SIN_PA:", p_value_23)

P-valor entre total_CON_PA_MECVV i total_SIN_PA: 0.6663533091466043


## Index del MNA

Primer convertiré la columna a tipus numèric ja que els valors que conté són de tipus objecte i amb aquest tipus no puc calcular la mitjana ni la desviació estàndard.

In [1564]:
CON_PA.loc[:, 'mna_resultats'] = pd.to_numeric(CON_PA['mna_resultats'], errors='coerce')
CON_PA_MECVV.loc[:, 'mna_resultats'] = pd.to_numeric(CON_PA_MECVV['mna_resultats'], errors='coerce')
SIN_PA.loc[:, 'mna_resultats'] = pd.to_numeric(SIN_PA['mna_resultats'], errors='coerce')

### Mitjana

In [1565]:
promedio_mna = CON_PA['mna_resultats'].mean()
desviacion_estandar_mna = CON_PA['mna_resultats'].std()

resultado = f"{promedio_mna:.2f} ± {desviacion_estandar_mna:.2f}"
print("Mitjana ± Desviació Estándard del mna:", resultado)

Mitjana ± Desviació Estándard del mna: 0.00 ± 0.00


In [1566]:
promedio_mna = CON_PA_MECVV['mna_resultats'].mean()
desviacion_estandar_mna = CON_PA_MECVV['mna_resultats'].std()

resultado = f"{promedio_mna:.2f} ± {desviacion_estandar_mna:.2f}"
print("Mitjana ± Desviació Estándard del mna:", resultado)

Mitjana ± Desviació Estándard del mna: 0.00 ± nan


In [1567]:
promedio_mna = SIN_PA['mna_resultats'].mean()
desviacion_estandar_mna = SIN_PA['mna_resultats'].std()

resultado = f"{promedio_mna:.2f} ± {desviacion_estandar_mna:.2f}"
print("Mitjana ± Desviació Estàndard del mna:", resultado)

Mitjana ± Desviació Estàndard del mna: 3.05 ± 5.92


#### Pvalor

##### Normalitat

In [1568]:
conpanone = CON_PA[CON_PA['mna_resultats'].notna()]
stat, p_value = shapiro(conpanone)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El número de files segueix una distribució normal.


In [ ]:
pa2none = CON_PA_MECVV[CON_PA_MECVV['mna_resultats'].notna()]
stat, p_value = shapiro(pa2none)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [1294]:
pa3none = SIN_PA[SIN_PA['mna_resultats'].notna()]
stat, p_value = shapiro(pa3none)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = nan
Valor p = nan
El número de files segueix una distribució normal.


In [1295]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['mna_resultats'], CON_PA_MECVV['mna_resultats'], equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: nan


In [1296]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['mna_resultats'], SIN_PA['mna_resultats'], equal_var=False)
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: nan


In [1297]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['mna_resultats'], SIN_PA['mna_resultats'], equal_var=False)
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: nan


### Valors de malnutrició

##### CON_PA

In [1298]:
norm_CON_PA = CON_PA['mna_resultats'].between(24, 30)
Percentatge_norm_CON_PA = norm_CON_PA.mean() * 100
num_observaciones_norm_CON_PA = norm_CON_PA.sum()

print(f"Percentatge de norm_CON_PA: {Percentatge_norm_CON_PA:.2f}%")
print(f"Nombre d'observacions en norm_CON_PA: {num_observaciones_norm_CON_PA}")

Percentatge de norm_CON_PA: 0.00%
Nombre d'observacions en norm_CON_PA: 0


In [1299]:
malnutr_CON_PA = CON_PA['mna_resultats'].between(17, 23.5)
Percentatge_malnutr_CON_PA = malnutr_CON_PA.mean() * 100
num_observaciones_malnutr_CON_PA = norm_CON_PA.sum()

print(f"Percentatge de malnutr_CON_PA: {Percentatge_malnutr_CON_PA:.2f}%")
print(f"Nombre d'observacions en norm_CON_PA: {num_observaciones_malnutr_CON_PA}")

Percentatge de malnutr_CON_PA: 0.00%
Nombre d'observacions en norm_CON_PA: 0


In [1300]:
malnut_CON_PA = CON_PA['mna_resultats'] < 17
Percentatge_malnut_CON_PA = malnut_CON_PA.mean() * 100
num_observaciones_malnut_CON_PA = malnut_CON_PA.sum()

print(f"Percentatge de malnut_CON_PA: {Percentatge_malnut_CON_PA:.2f}%")
print(f"Nombre d'observacions en malnut_CON_PA: {num_observaciones_malnut_CON_PA}")

Percentatge de malnut_CON_PA: 23.53%
Nombre d'observacions en malnut_CON_PA: 4


##### CON_PA_MECVV

In [1301]:
norm_CON_PA_MECVV = CON_PA_MECVV['mna_resultats'].between(24, 30)
Percentatge_norm_CON_PA_MECVV = norm_CON_PA_MECVV.mean() * 100
num_observaciones_norm_CON_PA_MECVV = norm_CON_PA_MECVV.sum()

print(f"Percentatge de norm_CON_PA_MECVV: {Percentatge_norm_CON_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en norm_CON_PA_MECVV: {num_observaciones_norm_CON_PA_MECVV}")

Percentatge de norm_CON_PA_MECVV: 0.00%
Nombre d'observacions en norm_CON_PA_MECVV: 0


In [1302]:
malnutr_CON_PA_MECVV = CON_PA_MECVV['mna_resultats'].between(17, 23.5)
Percentatge_malnutr_CON_PA_MECVV = malnutr_CON_PA_MECVV.mean() * 100
num_observaciones_malnutr_CON_PA_MECVV = norm_CON_PA_MECVV.sum()

print(f"Percentatge de malnutr_CON_PA_MECVV: {Percentatge_malnutr_CON_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en norm_CON_PA_MECVV: {num_observaciones_malnutr_CON_PA_MECVV}")

Percentatge de malnutr_CON_PA_MECVV: 0.00%
Nombre d'observacions en norm_CON_PA_MECVV: 0


In [1303]:
malnut_CON_PA_MECVV = CON_PA_MECVV['mna_resultats'] < 17
Percentatge_malnut_CON_PA_MECVV = malnut_CON_PA_MECVV.mean() * 100
num_observaciones_malnut_CON_PA_MECVV = malnut_CON_PA_MECVV.sum()

print(f"Percentatge de malnut_CON_PA_MECVV: {Percentatge_malnut_CON_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en malnut_CON_PA_MECVV: {num_observaciones_malnut_CON_PA_MECVV}")

Percentatge de malnut_CON_PA_MECVV: 10.00%
Nombre d'observacions en malnut_CON_PA_MECVV: 1


##### SIN_PA

In [1304]:
norm_SIN_PA = SIN_PA['mna_resultats'].between(24, 30)
Percentatge_norm_SIN_PA = norm_SIN_PA.mean() * 100
num_observaciones_norm_SIN_PA = norm_SIN_PA.sum()

print(f"Percentatge de norm_SIN_PA: {Percentatge_norm_SIN_PA:.2f}%")
print(f"Nombre d'observacions en norm_SIN_PA: {num_observaciones_norm_SIN_PA}")

Percentatge de norm_SIN_PA: 0.00%
Nombre d'observacions en norm_SIN_PA: 0


In [1305]:
malnutr_SIN_PA = SIN_PA['mna_resultats'].between(17, 23.5)
Percentatge_malnutr_SIN_PA = malnutr_SIN_PA.mean() * 100
num_observaciones_malnutr_SIN_PA = malnutr_SIN_PA.sum()

print(f"Percentatge de malnutr_SIN_PA: {Percentatge_malnutr_SIN_PA:.2f}%")
print(f"Nombre d'observacions en malnutr_SIN_PA: {num_observaciones_malnutr_SIN_PA}")

Percentatge de malnutr_SIN_PA: 2.04%
Nombre d'observacions en malnutr_SIN_PA: 1


In [1306]:
malnut_SIN_PA = SIN_PA['mna_resultats'] < 17
Percentatge_malnut_SIN_PA = malnut_SIN_PA.mean() * 100
num_observaciones_malnut_SIN_PA = malnut_SIN_PA.sum()

print(f"Percentatge de malnut_SIN_PA: {Percentatge_malnut_SIN_PA:.2f}%")
print(f"Nombre d'observacions en malnut_SIN_PA: {num_observaciones_malnut_SIN_PA}")

Percentatge de malnut_SIN_PA: 18.37%
Nombre d'observacions en malnut_SIN_PA: 9


#### Pvalor

##### Normal

In [1307]:
t_stat_12, p_val_12 = mannwhitneyu(CON_PA['mna_resultats'].between(24, 30), CON_PA_MECVV['mna_resultats'].between(24, 30), alternative='two-sided')
print("P-valor entre norm_CON_PA i norm_CON_PA_MECVV:", p_val_12)

P-valor entre norm_CON_PA i norm_CON_PA_MECVV: 1.0


In [1308]:
t_stat_13, p_val_13 = mannwhitneyu(CON_PA['mna_resultats'].between(24, 30), SIN_PA['mna_resultats'].between(24, 30), alternative='two-sided')
print("P-valor entre norm_CON_PA i norm_SIN_PA:", p_val_13)

P-valor entre norm_CON_PA i norm_SIN_PA: 1.0


In [1309]:
t_stat_23, p_val_23 = mannwhitneyu(CON_PA_MECVV['mna_resultats'].between(24, 30), SIN_PA['mna_resultats'].between(24, 30), alternative='two-sided')
print("P-valor entre norm_CON_PA_MECVV i norm_SIN_PA:", p_val_23)

P-valor entre norm_CON_PA_MECVV i norm_SIN_PA: 1.0


##### Risc de malnutrició

In [1310]:
t_stat_12, p_val_12 = mannwhitneyu(CON_PA['mna_resultats'].between(17, 23.5), CON_PA_MECVV['mna_resultats'].between(17, 23.5), alternative='two-sided')
print("P-valor entre malnutr_CON_PA i malnutr_CON_PA_MECVV:", p_val_12)

P-valor entre malnutr_CON_PA i malnutr_CON_PA_MECVV: 1.0


In [1311]:
t_stat_13, p_val_13 = mannwhitneyu(CON_PA['mna_resultats'].between(17, 23.5), SIN_PA['mna_resultats'].between(17, 23.5), alternative='two-sided')
print("P-valor entre malnutr_CON_PA i malnutr_SIN_PA:", p_val_13)

P-valor entre malnutr_CON_PA i malnutr_SIN_PA: 0.5793276078637376


In [1312]:
t_stat_23, p_val_23 = mannwhitneyu(CON_PA_MECVV['mna_resultats'].between(17, 23.5), SIN_PA['mna_resultats'].between(17, 23.5), alternative='two-sided')
print("P-valor entre malnutr_CON_PA_MECVV i malnutr_SIN_PA:", p_val_23)

P-valor entre malnutr_CON_PA_MECVV i malnutr_SIN_PA: 0.6843175494727258


##### Malnutrició

In [1313]:
t_stat_12, p_val_12 = mannwhitneyu(CON_PA['mna_resultats'] <17 , 
                                CON_PA_MECVV['mna_resultats'] < 17, alternative='two-sided')
print("P-valor entre malnut_CON_PA i norm_CON_PA_MECVV:", p_val_12)

P-valor entre malnut_CON_PA i norm_CON_PA_MECVV: 0.4120280546315189


In [1314]:
t_stat_13, p_val_13 = mannwhitneyu(CON_PA['mna_resultats'] <17 , 
                                SIN_PA['mna_resultats'] < 17, alternative='two-sided')
print("P-valor entre malnut_CON_PA i malnut_SIN_PA:", p_val_13)

P-valor entre malnut_CON_PA i malnut_SIN_PA: 0.6549000779898075


In [1315]:
t_stat_23, p_val_23 = mannwhitneyu(CON_PA_MECVV['mna_resultats'] <17 , 
                                SIN_PA['mna_resultats'] < 17, alternative='two-sided')
print("P-valor entre malnut_CON_PA_MECVV i malnut_SIN_PA:", p_val_23)

P-valor entre malnut_CON_PA_MECVV i malnut_SIN_PA: 0.5341419377112933


## Index de l'EMINA

### Mitjana

In [1316]:
promedio_emina = CON_PA['EMINA_sumatorios_comparados'].mean()
desviacion_estandar_emina = CON_PA['EMINA_sumatorios_comparados'].std()

resultado = f"{promedio_emina:.2f} ± {desviacion_estandar_emina:.2f}"
print("Mitjana ± Desviació Estàndard de l'emina:", resultado)

Mitjana ± Desviació Estàndard de la emina: 7.94 ± 3.21


In [1317]:
promedio_emina = CON_PA_MECVV['EMINA_sumatorios_comparados'].mean()
desviacion_estandar_emina = CON_PA_MECVV['EMINA_sumatorios_comparados'].std()

resultado = f"{promedio_emina:.2f} ± {desviacion_estandar_emina:.2f}"
print("Mitjana ± Desviació Estàndard de l'emina:", resultado)

Mitjana ± Desviació Estàndard de la emina: 7.60 ± 2.01


In [1493]:
promedio_emina = SIN_PA['EMINA_sumatorios_comparados'].mean()
desviacion_estandar_emina = SIN_PA['EMINA_sumatorios_comparados'].std()

resultado = f"{promedio_emina:.2f} ± {desviacion_estandar_emina:.2f}"
print("Mitjana ± Desviació Estàndard de l'emina:", resultado)

Mitjana ± Desviació Estàndard de l'emina: 5.64 ± 4.08


#### Pvalor

##### Normalitat

In [ ]:
stat, p_value = shapiro(CON_PA['EMINA_sumatorios_comparados']).dropna()
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
stat, p_value = shapiro(CON_PA_MECVV['EMINA_sumatorios_comparados']).dropna()
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
stat, p_value = shapiro(SIN_PA['EMINA_sumatorios_comparados']).dropna()
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [1322]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['EMINA_sumatorios_comparados'], CON_PA_MECVV['EMINA_sumatorios_comparados'], equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: 0.7372065144531302


In [1323]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['EMINA_sumatorios_comparados'], SIN_PA['EMINA_sumatorios_comparados'], equal_var=False)
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: nan


In [1324]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['EMINA_sumatorios_comparados'], SIN_PA['EMINA_sumatorios_comparados'], equal_var=False)
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: nan


### Valors EMINA

##### CON_PA

In [1325]:
lowr_CON_PA = CON_PA['EMINA_sumatorios_comparados'].between(0, 5)
Percentatge_lowr_CON_PA = lowr_CON_PA.mean() * 100
num_observaciones_lowr_CON_PA = lowr_CON_PA.sum()

print(f"Percentatge de lowr_CON_PA: {Percentatge_lowr_CON_PA:.2f}%")
print(f"Nombre d'observacions en lowr_CON_PA: {num_observaciones_lowr_CON_PA}")

Percentatge de lowr_CON_PA: 17.65%
Nombre d'observacions en lowr_CON_PA: 3


In [1326]:
modr_CON_PA = CON_PA['EMINA_sumatorios_comparados'].between(6, 10)
Percentatge_modr_CON_PA = modr_CON_PA.mean() * 100
num_observaciones_modr_CON_PA = modr_CON_PA.sum()

print(f"Percentatge de modr_CON_PA: {Percentatge_modr_CON_PA:.2f}%")
print(f"Nombre d'observacions en modr_CON_PA: {num_observaciones_modr_CON_PA}")

Percentatge de modr_CON_PA: 64.71%
Nombre d'observacions en modr_CON_PA: 11


In [1327]:
highr_CON_PA = CON_PA['EMINA_sumatorios_comparados'].between(11, 15)
Percentatge_highr_CON_PA = highr_CON_PA.mean() * 100
num_observaciones_highr_CON_PA = highr_CON_PA.sum()

print(f"Percentatge de highr_CON_PA: {Percentatge_highr_CON_PA:.2f}%")
print(f"Nombre d'observacions en highr_CON_PA: {num_observaciones_highr_CON_PA}")

Percentatge de highr_CON_PA: 17.65%
Nombre d'observacions en highr_CON_PA: 3


##### CON_PA_MECVV

In [1328]:
lowr_CON_PA_MECVV = CON_PA_MECVV['EMINA_sumatorios_comparados'].between(0, 5)
Percentatge_lowr_CON_PA_MECVV = lowr_CON_PA_MECVV.mean() * 100
num_observaciones_lowr_CON_PA_MECVV = lowr_CON_PA_MECVV.sum()

print(f"Percentatge de lowr_CON_PA_MECVV: {Percentatge_lowr_CON_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en lowr_CON_PA_MECVV: {num_observaciones_lowr_CON_PA_MECVV}")

Percentatge de lowr_CON_PA_MECVV: 20.00%
Nombre d'observacions en lowr_CON_PA_MECVV: 2


In [1329]:
modr_CON_PA_MECVV = CON_PA_MECVV['EMINA_sumatorios_comparados'].between(6, 10)
Percentatge_modr_CON_PA_MECVV = modr_CON_PA_MECVV.mean() * 100
num_observaciones_modr_CON_PA_MECVV = modr_CON_PA_MECVV.sum()

print(f"Percentatge de modr_CON_PA_MECVV: {Percentatge_modr_CON_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en modr_CON_PA_MECVV: {num_observaciones_modr_CON_PA_MECVV}")

Percentatge de modr_CON_PA_MECVV: 80.00%
Nombre d'observacions en modr_CON_PA_MECVV: 8


In [1330]:
highr_CON_PA_MECVV = CON_PA_MECVV['EMINA_sumatorios_comparados'].between(11, 15)
Percentatge_highr_CON_PA_MECVV = highr_CON_PA_MECVV.mean() * 100
num_observaciones_highr_CON_PA_MECVV = highr_CON_PA_MECVV.sum()

print(f"Percentatge de highr_CON_PA_MECVV: {Percentatge_highr_CON_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en highr_CON_PA_MECVV: {num_observaciones_highr_CON_PA_MECVV}")

Percentatge de highr_CON_PA_MECVV: 0.00%
Nombre d'observacions en highr_CON_PA_MECVV: 0


##### SIN_PA

In [1331]:
lowr_SIN_PA = SIN_PA['EMINA_sumatorios_comparados'].between(0, 5)
Percentatge_lowr_SIN_PA = lowr_SIN_PA.mean() * 100
num_observaciones_lowr_SIN_PA = lowr_SIN_PA.sum()

print(f"Percentatge de lowr_SIN_PA: {Percentatge_lowr_SIN_PA:.2f}%")
print(f"Nombre d'observacions en lowr_SIN_PA: {num_observaciones_lowr_SIN_PA}")

Percentatge de lowr_SIN_PA: 44.90%
Nombre d'observacions en lowr_SIN_PA: 22


In [1332]:
modr_SIN_PA = SIN_PA['EMINA_sumatorios_comparados'].between(6, 10)
Percentatge_modr_SIN_PA = modr_SIN_PA.mean() * 100
num_observaciones_modr_SIN_PA = modr_SIN_PA.sum()

print(f"Percentatge de modr_SIN_PA: {Percentatge_modr_SIN_PA:.2f}%")
print(f"Nombre d'observacions en modr_SIN_PA: {num_observaciones_modr_SIN_PA}")

Percentatge de modr_SIN_PA: 40.82%
Nombre d'observacions en modr_SIN_PA: 20


In [1333]:
highr_SIN_PA = SIN_PA['EMINA_sumatorios_comparados'].between(11, 15)
Percentatge_highr_SIN_PA = highr_SIN_PA.mean() * 100
num_observaciones_highr_SIN_PA = highr_SIN_PA.sum()

print(f"Percentatge de highr_SIN_PA: {Percentatge_highr_SIN_PA:.2f}%")
print(f"Nombre d'observacions en highr_SIN_PA: {num_observaciones_highr_SIN_PA}")

Percentatge de highr_SIN_PA: 10.20%
Nombre d'observacions en highr_SIN_PA: 5


#### Pvalor 

##### Risc baix

In [1334]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['EMINA_sumatorios_comparados'].between(0,5), CON_PA_MECVV['EMINA_sumatorios_comparados'].between(0,5), equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: 0.8874457485314164


In [1335]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['EMINA_sumatorios_comparados'].between(0,5), SIN_PA['EMINA_sumatorios_comparados'].between(0,5), equal_var=False)
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: 0.028470032037651503


In [1336]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['EMINA_sumatorios_comparados'].between(0,5), SIN_PA['EMINA_sumatorios_comparados'].between(0,5), equal_var=False)
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: 0.12129243251471264


##### Risc moderat

In [1337]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['EMINA_sumatorios_comparados'].between(6,10), CON_PA_MECVV['EMINA_sumatorios_comparados'].between(6,10), equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: 0.40237952012988454


In [1338]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['EMINA_sumatorios_comparados'].between(6,10), SIN_PA['EMINA_sumatorios_comparados'].between(6,10), equal_var=False)
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: 0.09654829012396034


In [1339]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['EMINA_sumatorios_comparados'].between(6,10), SIN_PA['EMINA_sumatorios_comparados'].between(6,10), equal_var=False)
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: 0.02066325652755892


##### Risc alt

In [1340]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['EMINA_sumatorios_comparados'].between(11,15), CON_PA_MECVV['EMINA_sumatorios_comparados'].between(11,15), equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: 0.08262239198015026


In [1341]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['EMINA_sumatorios_comparados'].between(11,15), SIN_PA['EMINA_sumatorios_comparados'].between(11,15), equal_var=False)
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: 0.48486212369214565


In [1342]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['EMINA_sumatorios_comparados'].between(11,15), SIN_PA['EMINA_sumatorios_comparados'].between(11,15), equal_var=False)
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: 0.02374418239053046


## Index de la canadenca 

### Mitjana

In [1343]:
promedio_canad = CON_PA['Canadenca_sumatorios_comparados'].mean()
desviacion_estandar_canad = CON_PA['Canadenca_sumatorios_comparados'].std()

resultado = f"{promedio_canad:.2f} ± {desviacion_estandar_canad:.2f}"
print("Mitjana ± Desviació Estàndard de la canadenca:", resultado)

Mitjana ± Desviació Estàndard de la canadenca: 7.00 ± nan


In [1344]:
promedio_canad = CON_PA_MECVV['Canadenca_sumatorios_comparados'].mean()
desviacion_estandar_canad = CON_PA_MECVV['Canadenca_sumatorios_comparados'].std()

resultado = f"{promedio_canad:.2f} ± {desviacion_estandar_canad:.2f}"
print("Mitjana ± Desviació Estàndard de la canadenca:", resultado)

Mitjana ± Desviació Estàndard de la canadenca: 7.00 ± nan


In [1345]:
promedio_canad = SIN_PA['Canadenca_sumatorios_comparados'].mean()
desviacion_estandar_canad = SIN_PA['Canadenca_sumatorios_comparados'].std()

resultado = f"{promedio_canad:.2f} ± {desviacion_estandar_canad:.2f}"
print("Mitjana ± Desviació Estàndard de la canadenca:", resultado)

Mitjana ± Desviació Estàndard de la canadenca: 8.00 ± 1.41


#### Pvalor

##### Normalitat

In [ ]:
stat, p_value = shapiro(CON_PA['Canadenca_sumatorios_comparados']).dropna()
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
stat, p_value = shapiro(CON_PA_MECVV['Canadenca_sumatorios_comparados']).dropna()
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

In [ ]:
stat, p_value = shapiro(SIN_PA['Canadenca_sumatorios_comparados']).dropna()
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

#TODO: modificar esto si siguen normalidad

In [1346]:
t_stat_12, p_val_12 = mannwhitneyu(CON_PA['Canadenca_sumatorios_comparados'], CON_PA_MECVV['Canadenca_sumatorios_comparados'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: nan


In [1347]:
t_stat_13, p_val_13 = mannwhitneyu(CON_PA['Canadenca_sumatorios_comparados'], SIN_PA['Canadenca_sumatorios_comparados'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: nan


In [1348]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['Canadenca_sumatorios_comparados'], SIN_PA['Canadenca_sumatorios_comparados'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: nan


### Valors canadenca

##### CON_PA 

In [1349]:
mild_CON_PA = CON_PA['Canadenca_sumatorios_comparados'].between(10.5, 11.5)
Percentatge_mild_CON_PA = mild_CON_PA.mean() * 100
num_observaciones_mild_CON_PA = mild_CON_PA.sum()

print(f"Percentatge de mild_CON_PA: {Percentatge_mild_CON_PA:.2f}%")
print(f"Nombre d'observacions en mild_CON_PA: {num_observaciones_mild_CON_PA}")

Percentatge de mild_CON_PA: 0.00%
Nombre d'observacions en mild_CON_PA: 0


In [1350]:
modneu_CON_PA = CON_PA['Canadenca_sumatorios_comparados'].between(5,10)
Percentatge_modneu_CON_PA = modneu_CON_PA.mean() * 100
num_observaciones_modneu_CON_PA = modneu_CON_PA.sum()

print(f"Percentatge de modneu_CON_PA: {Percentatge_modneu_CON_PA:.2f}%")
print(f"Nombre d'observacions en modneu_CON_PA: {num_observaciones_modneu_CON_PA}")

Percentatge de modneu_CON_PA: 5.88%
Nombre d'observacions en modneu_CON_PA: 1


In [1351]:
sevneu_CON_PA = CON_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5)
Percentatge_sevneu_CON_PA = sevneu_CON_PA.mean() * 100
num_observaciones_sevneu_CON_PA = sevneu_CON_PA.sum()

print(f"Percentatge de sevneu_CON_PA: {Percentatge_sevneu_CON_PA:.2f}%")
print(f"Nombre d'observacions en sevneu_CON_PA: {num_observaciones_sevneu_CON_PA}")

Percentatge de sevneu_CON_PA: 0.00%
Nombre d'observacions en sevneu_CON_PA: 0


##### CON_PA_MECVV

In [1352]:
mild_CON_PA_MECVV = CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(10.5, 11.5)
Percentatge_mild_CON_PA_MECVV = mild_CON_PA_MECVV.mean() * 100
num_observaciones_mild_CON_PA_MECVV = mild_CON_PA_MECVV.sum()

print(f"Percentatge de mild_CON_PA_MECVV: {Percentatge_mild_CON_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en mild_CON_PA_MECVV: {num_observaciones_mild_CON_PA_MECVV}")

Percentatge de mild_CON_PA_MECVV: 0.00%
Nombre d'observacions en mild_CON_PA_MECVV: 0


In [1353]:
modneu_CON_PA_MECVV = CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(5,10)
Percentatge_modneu_CON_PA_MECVV = modneu_CON_PA_MECVV.mean() * 100
num_observaciones_modneu_CON_PA_MECVV = modneu_CON_PA_MECVV.sum()

print(f"Percentatge de modneu_CON_PA_MECVV: {Percentatge_modneu_CON_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en modneu_CON_PA_MECVV: {num_observaciones_modneu_CON_PA_MECVV}")

Percentatge de modneu_CON_PA_MECVV: 10.00%
Nombre d'observacions en modneu_CON_PA_MECVV: 1


In [1354]:
sevneu_CON_PA_MECVV = CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(1.5, 4.5)
Percentatge_sevneu_CON_PA_MECVV = sevneu_CON_PA_MECVV.mean() * 100
num_observaciones_sevneu_CON_PA_MECVV = sevneu_CON_PA_MECVV.sum()

print(f"Percentatge de sevneu_CON_PA_MECVV: {Percentatge_sevneu_CON_PA_MECVV:.2f}%")
print(f"Nombre d'observacions en sevneu_CON_PA_MECVV: {num_observaciones_sevneu_CON_PA_MECVV}")

Percentatge de sevneu_CON_PA_MECVV: 0.00%
Nombre d'observacions en sevneu_CON_PA_MECVV: 0


##### SIN_PA

In [1355]:
mild_SIN_PA = SIN_PA['Canadenca_sumatorios_comparados'].between(10.5, 11.5)
Percentatge_mild_SIN_PA = mild_SIN_PA.mean() * 100
num_observaciones_mild_SIN_PA = mild_SIN_PA.sum()

print(f"Percentatge de mild_SIN_PA: {Percentatge_mild_SIN_PA:.2f}%")
print(f"Nombre d'observacions en mild_SIN_PA: {num_observaciones_mild_SIN_PA}")

Percentatge de mild_SIN_PA: 0.00%
Nombre d'observacions en mild_SIN_PA: 0


In [1356]:
modneu_SIN_PA = SIN_PA['Canadenca_sumatorios_comparados'].between(5,10)
Percentatge_modneu_SIN_PA = modneu_SIN_PA.mean() * 100
num_observaciones_modneu_SIN_PA = modneu_SIN_PA.sum()

print(f"Percentatge de modneu_SIN_PA: {Percentatge_modneu_SIN_PA:.2f}%")
print(f"Nombre d'observacions en modneu_SIN_PA: {num_observaciones_modneu_SIN_PA}")

Percentatge de modneu_SIN_PA: 4.08%
Nombre d'observacions en modneu_SIN_PA: 2


In [1357]:
sevneu_SIN_PA = SIN_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5)
Percentatge_sevneu_SIN_PA = sevneu_SIN_PA.mean() * 100
num_observaciones_sevneu_SIN_PA = sevneu_SIN_PA.sum()

print(f"Percentatge de sevneu_SIN_PA: {Percentatge_sevneu_SIN_PA:.2f}%")
print(f"Nombre d'observacions en sevneu_SIN_PA: {num_observaciones_sevneu_SIN_PA}")

Percentatge de sevneu_SIN_PA: 0.00%
Nombre d'observacions en sevneu_SIN_PA: 0


#### Pvalor

##### Dèficit neurològic lleuger (mild)

In [1358]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['Canadenca_sumatorios_comparados'].between(10.5,11.5), CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(10.5,11.5), equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: nan


In [1359]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['Canadenca_sumatorios_comparados'].between(10.5,11.5), SIN_PA['Canadenca_sumatorios_comparados'].between(10.5,11.5), equal_var=False)
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: nan


In [1360]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(10.5,11.5), SIN_PA['Canadenca_sumatorios_comparados'].between(10.5,11.5), equal_var=False)
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: nan


##### Dèficit neurològic moderat

In [1361]:
t_stat_12, p_val_12 = mannwhitneyu(CON_PA['Canadenca_sumatorios_comparados'].between(5,10), CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(5, 10), alternative='less')
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: 0.3700051180291794


In [1362]:
t_stat_13, p_val_13 = mannwhitneyu(CON_PA['Canadenca_sumatorios_comparados'].between(5,10), SIN_PA['Canadenca_sumatorios_comparados'].between(5, 10), alternative='less')
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: 0.6274490868971419


In [1363]:
t_stat_23, p_val_23 = mannwhitneyu(CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(5,10), SIN_PA['Canadenca_sumatorios_comparados'].between(5, 10), alternative='less')
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: 0.7870742763184819


##### Dèficit neurològic greu (sever)

In [1364]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['Canadenca_sumatorios_comparados'].between(1.5,4.5), CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(1.5,4.5), equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: nan


In [1365]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5), SIN_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5), equal_var=False)
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: nan


In [1366]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['Canadenca_sumatorios_comparados'].between(1.5,4.5), SIN_PA['Canadenca_sumatorios_comparados'].between(1.5, 4.5), equal_var=False)
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: nan


## Infeccions respiratòries

##### CON_PA 

In [1367]:
mainresp_CON_PA= (CON_PA['Main_respiratory_infections_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de infeccions respiratòries principals : {mainresp_CON_PA:.2f}%")

Percentatge de infecciones respiratorias principales : 41.18%


In [1368]:
LRTI_CON_PA= (CON_PA['LRTI_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de LRTI: {LRTI_CON_PA:.2f}%")

Percentatge de LRTI: 17.65%


In [1369]:
COPD_CON_PA= (CON_PA['COPD_exacerbations_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de exacerbacions de COPD: {COPD_CON_PA:.2f}%")

Percentatge de exacerbaciones de COPD: 29.41%


In [1370]:
fibro_CON_PA= (CON_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de fibrosi pulmonar i fibrotòrax: {fibro_CON_PA:.2f}%")

Percentatge de fibrosis pulmonar i fibrotorax: 0.00%


##### CON_PA_MECVV

In [1371]:
mainresp_CON_PA_MECVV= (CON_PA_MECVV['Main_respiratory_infections_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de infeccions respiratòries principals : {mainresp_CON_PA_MECVV:.2f}%")

Percentatge de infecciones respiratorias principales : 50.00%


In [1372]:
LRTI_CON_PA_MECVV= (CON_PA_MECVV['LRTI_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de LRTI: {LRTI_CON_PA_MECVV:.2f}%")

Percentatge de LRTI: 20.00%


In [1373]:
COPD_CON_PA_MECVV = (CON_PA_MECVV['COPD_exacerbations_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de exacerbacions de COPD: {COPD_CON_PA_MECVV:.2f}%")

Percentatge de exacerbaciones de COPD: 30.00%


In [1374]:
fibro_CON_PA_MECVV = (CON_PA_MECVV['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de fibrosi pulmonar i fibrotòrax: {fibro_CON_PA_MECVV:.2f}%")

Percentatge de fibrosis pulmonar i fibrotorax: 0.00%


##### SIN_PA

In [1375]:
mainresp_SIN_PA= (SIN_PA['Main_respiratory_infections_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de infeccions respiratòries principals : {mainresp_SIN_PA:.2f}%")

Percentatge de infecciones respiratorias principales : 42.86%


In [1376]:
LRTI_SIN_PA= (SIN_PA['LRTI_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de LRTI: {LRTI_SIN_PA:.2f}%")

Percentatge de LRTI: 22.45%


In [1377]:
COPD_SIN_PA= (SIN_PA['COPD_exacerbations_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de exacerbacions de COPD: {COPD_SIN_PA:.2f}%")

Percentatge de exacerbaciones de COPD: 32.65%


In [1378]:
fibro_SIN_PA= (SIN_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0).mean() * 100
print(f"Percentatge de fibrosi pulmonar i fibrotòrax: {fibro_SIN_PA:.2f}%")

Percentatge de fibrosis pulmonar i fibrotorax: 0.00%


#### Pvalor

##### Normalitat

In [1379]:
stat, p_value = shapiro(CON_PA['Main_respiratory_infections_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.6317
Valor p = 0.00002144
El número de files no segueix una distribució normal.


In [1380]:
stat, p_value = shapiro(CON_PA['LRTI_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.4702
Valor p = 0.00000073
El número de files no segueix una distribució normal.


In [1381]:
stat, p_value = shapiro(CON_PA['COPD_exacerbations_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.5785
Valor p = 0.00000650
El número de files no segueix una distribució normal.


In [1382]:
stat, p_value = shapiro(CON_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 1.0000
Valor p = 1.00000000
El número de files segueix una distribució normal.


C:\Users\maria\PycharmProjects\practicum\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


Usaré Mann-Whitney cuando tenga que comparar contra CON_PA

In [1383]:
stat, p_value = shapiro(CON_PA_MECVV['Main_respiratory_infections_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.6553
Valor p = 0.00025396
El número de files no segueix una distribució normal.


In [1384]:
stat, p_value = shapiro(CON_PA_MECVV['LRTI_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.5093
Valor p = 0.00000467
El número de files no segueix una distribució normal.


In [1385]:
stat, p_value = shapiro(CON_PA_MECVV['COPD_exacerbations_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.5942
Valor p = 0.00004713
El número de files no segueix una distribució normal.


In [1386]:
stat, p_value = shapiro(CON_PA_MECVV['Pulmonary_fibrosis_fibrotorax_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 1.0000
Valor p = 1.00000000
El número de files segueix una distribució normal.


Usaré también Mann-Whitney

In [1387]:
stat, p_value = shapiro(SIN_PA['Main_respiratory_infections_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.6294
Valor p = 0.00000000
El número de files no segueix una distribució normal.


In [1388]:
stat, p_value = shapiro(SIN_PA['LRTI_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.5158
Valor p = 0.00000000
El número de files no segueix una distribució normal.


In [1389]:
stat, p_value = shapiro(SIN_PA['COPD_exacerbations_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.5916
Valor p = 0.00000000
El número de files no segueix una distribució normal.


In [1390]:
stat, p_value = shapiro(SIN_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 1.0000
Valor p = 1.00000000
El número de files segueix una distribució normal.


##### Infeccions respiratòries principals 

In [1391]:
stat_12, p_val_12 = mannwhitneyu(CON_PA['Main_respiratory_infections_diagnosticada'] == 1.0, CON_PA_MECVV['Main_respiratory_infections_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: 0.6832089509594992


In [1392]:
stat_13, p_val_13 = mannwhitneyu(CON_PA['Main_respiratory_infections_diagnosticada'] == 1.0, SIN_PA['Main_respiratory_infections_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: 0.9113557832989896


In [1393]:
stat_23, p_val_23 = mannwhitneyu(CON_PA_MECVV['Main_respiratory_infections_diagnosticada'] == 1.0, SIN_PA['Main_respiratory_infections_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: 0.6896369959432476


##### LRTI

In [1394]:
stat_12, p_val_12 = mannwhitneyu(CON_PA['LRTI_diagnosticada'] == 1.0, CON_PA_MECVV['LRTI_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: 0.9109318987037512


In [1395]:
stat_13, p_val_13 = mannwhitneyu(CON_PA['LRTI_diagnosticada'] == 1.0, SIN_PA['LRTI_diagnosticada'] == 1.0,alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: 0.6863825106164784


In [1396]:
stat_23, p_val_23 = mannwhitneyu(CON_PA_MECVV['LRTI_diagnosticada'] == 1.0, SIN_PA['LRTI_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: 0.8770107151211659


##### Exacerbacions de COPD

In [1397]:
stat_12, p_val_12 = mannwhitneyu(CON_PA['COPD_exacerbations_diagnosticada'] == 1.0, CON_PA_MECVV['COPD_exacerbations_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: 1.0


In [1398]:
stat_13, p_val_13 = mannwhitneyu(CON_PA['COPD_exacerbations_diagnosticada'] == 1.0, SIN_PA['COPD_exacerbations_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: 0.8132303236327403


In [1399]:
stat_23, p_val_23 = mannwhitneyu(CON_PA_MECVV['COPD_exacerbations_diagnosticada'] == 1.0, SIN_PA['COPD_exacerbations_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: 0.8809554446418177


##### Fibrosi pulmonar i fibrotòrax

In [1400]:
stat_12, p_val_12 = mannwhitneyu(CON_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, CON_PA_MECVV['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: 1.0


In [1401]:
stat_13, p_val_13 = mannwhitneyu(CON_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, SIN_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: 1.0


In [1402]:
stat_23, p_val_23 = mannwhitneyu(CON_PA_MECVV['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, SIN_PA['Pulmonary_fibrosis_fibrotorax_diagnosticada'] == 1.0, alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: 1.0


## Resultats de laboratori

Les columnes generades a partir d'aquests resultats contenen valors tipus objecte, per tant, primer els transformaré a tipus numèric per poder realitzar les operacions que neccessitaré fer.

### Albúmina

In [1403]:
CON_PA.loc[:, 'albumina'] = pd.to_numeric(CON_PA['albumina'], errors='coerce')

promedio_alb = CON_PA['albumina'].mean()
desviacion_estandar_alb = CON_PA['albumina'].std()

resultado = f"{promedio_alb:.2f} ± {desviacion_estandar_alb:.2f}"
print("Mitjana ± Desviació Estàndard de la albumina:", resultado)

Mitjana ± Desviació Estàndard de la albumina: 3.34 ± 0.38


In [1404]:
CON_PA_MECVV.loc[:, 'albumina'] = pd.to_numeric(CON_PA_MECVV['albumina'], errors='coerce')

promedio_alb = CON_PA_MECVV['albumina'].mean()
desviacion_estandar_alb = CON_PA_MECVV['albumina'].std()

resultado = f"{promedio_alb:.2f} ± {desviacion_estandar_alb:.2f}"
print("Mitjana ± Desviació Estàndard de la albumina:", resultado)

Mitjana ± Desviació Estàndard de la albumina: 3.47 ± 0.55


In [1405]:
SIN_PA.loc[:, 'albumina'] = pd.to_numeric(SIN_PA['albumina'], errors='coerce')

promedio_alb = SIN_PA['albumina'].mean()
desviacion_estandar_alb = SIN_PA['albumina'].std()

resultado = f"{promedio_alb:.2f} ± {desviacion_estandar_alb:.2f}"
print("Mitjana ± Desviació Estàndard de la albumina:", resultado)

Mitjana ± Desviació Estàndard de la albumina: 3.58 ± 0.64


#### Pvalor

##### Normalitat

In [1406]:
albpa = CON_PA['albumina'].dropna()
stat, p_value = shapiro(albpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9311
Valor p = 0.56068160
El número de files segueix una distribució normal.


In [1407]:
albconpamecvv = CON_PA_MECVV['albumina'].dropna()
stat, p_value = shapiro(albconpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9973
Valor p = 0.89985028
El número de files segueix una distribució normal.


In [1408]:
albsinpa= SIN_PA['albumina'].dropna()
stat, p_value = shapiro(albsinpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9778
Valor p = 0.92442182
El número de files segueix una distribució normal.


In [1409]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['albumina'], CON_PA_MECVV['albumina'], equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: nan


In [1410]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['albumina'], SIN_PA['albumina'], equal_var=False)
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: nan


In [1411]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['albumina'], SIN_PA['albumina'], equal_var=False)
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: nan


### Proteïnes totals

In [1412]:
CON_PA.loc[:, 'proteinas totales'] = pd.to_numeric(CON_PA['proteinas totales'], errors='coerce')

promedio_prot = CON_PA['proteinas totales'].mean()
desviacion_estandar_prot = CON_PA['proteinas totales'].std()

resultado = f"{promedio_prot:.2f} ± {desviacion_estandar_prot:.2f}"
print("Mitjana ± Desviació Estàndard de las proteinas totales:", resultado)

Mitjana ± Desviació Estàndard de las proteinas totales: 6.03 ± 0.73


In [1413]:
CON_PA_MECVV.loc[:, 'proteinas totales'] = pd.to_numeric(CON_PA_MECVV['proteinas totales'], errors='coerce')

promedio_prot = CON_PA_MECVV['proteinas totales'].mean()
desviacion_estandar_prot = CON_PA_MECVV['proteinas totales'].std()

resultado = f"{promedio_prot:.2f} ± {desviacion_estandar_prot:.2f}"
print("Mitjana ± Desviació Estàndard de las proteinas totales:", resultado)

Mitjana ± Desviació Estàndard de las proteinas totales: 6.47 ± 0.76


In [1414]:
SIN_PA.loc[:, 'proteinas totales'] = pd.to_numeric(SIN_PA['proteinas totales'], errors='coerce')

promedio_prot = SIN_PA['proteinas totales'].mean()
desviacion_estandar_prot = SIN_PA['proteinas totales'].std()

resultado = f"{promedio_prot:.2f} ± {desviacion_estandar_prot:.2f}"
print("Mitjana ± Desviació Estàndard de las proteinas totales:", resultado)

Mitjana ± Desviació Estàndard de las proteinas totales: 6.12 ± 0.72


#### Pvalor

##### Normalitat

In [1415]:
protconpa=CON_PA['proteinas totales'].dropna()
stat, p_value = shapiro(protconpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9780
Valor p = 0.95346065
El número de files segueix una distribució normal.


In [1416]:
protconpamecvv=CON_PA_MECVV['proteinas totales'].dropna()
stat, p_value = shapiro(protconpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9975
Valor p = 0.99159177
El número de files segueix una distribució normal.


In [1417]:
protsinpa=SIN_PA['proteinas totales'].dropna()
stat, p_value = shapiro(protsinpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9753
Valor p = 0.86021925
El número de files segueix una distribució normal.


In [1418]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['proteinas totales'], CON_PA_MECVV['proteinas totales'], equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: nan


In [1419]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['proteinas totales'], SIN_PA['proteinas totales'], equal_var=False)
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: nan


In [1420]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['proteinas totales'], SIN_PA['proteinas totales'], equal_var=False)
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: nan


### Hemoglobina

In [1421]:
CON_PA.loc[:, 'Hb'] = pd.to_numeric(CON_PA['Hb'], errors='coerce')

promedio_hb = CON_PA['Hb'].mean()
desviacion_estandar_hb = CON_PA['Hb'].std()

resultado = f"{promedio_hb:.2f} ± {desviacion_estandar_hb:.2f}"
print("Mitjana ± Desviació Estàndard de la hemoglobina:", resultado)

Mitjana ± Desviació Estàndard de la hemoglobina: 12.34 ± 1.77


In [1422]:
CON_PA_MECVV.loc[:, 'Hb'] = pd.to_numeric(CON_PA_MECVV['Hb'], errors='coerce')

promedio_hb = CON_PA_MECVV['Hb'].mean()
desviacion_estandar_hb = CON_PA_MECVV['Hb'].std()

resultado = f"{promedio_hb:.2f} ± {desviacion_estandar_hb:.2f}"
print("Mitjana ± Desviació Estàndard de la hemoglobina:", resultado)

Mitjana ± Desviació Estàndard de la hemoglobina: 13.22 ± 1.40


In [1423]:
SIN_PA.loc[:, 'Hb'] = pd.to_numeric(SIN_PA['Hb'], errors='coerce')

promedio_hb = SIN_PA['Hb'].mean()
desviacion_estandar_hb = SIN_PA['Hb'].std()

resultado = f"{promedio_hb:.2f} ± {desviacion_estandar_hb:.2f}"
print("Mitjana ± Desviació Estàndard de la hemoglobina:", resultado)

Mitjana ± Desviació Estàndard de la hemoglobina: 13.22 ± 2.16


##### Pvalor

##### Normalitat

In [1424]:
hbconpa=CON_PA['Hb'].dropna()
stat, p_value = shapiro(hbconpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9304
Valor p = 0.48501101
El número de files segueix una distribució normal.


In [1425]:
hbconpamecvv=CON_PA_MECVV['Hb'].dropna()
stat, p_value = shapiro(hbconpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8592
Valor p = 0.25743995
El número de files segueix una distribució normal.


In [1426]:
hbsinpa=SIN_PA['Hb'].dropna()
stat, p_value = shapiro(hbsinpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9551
Valor p = 0.37239632
El número de files segueix una distribució normal.


In [1427]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['Hb'], CON_PA_MECVV['Hb'], equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: nan


In [1428]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['Hb'], SIN_PA['Hb'], equal_var=False)
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: nan


In [1429]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['Hb'], SIN_PA['Hb'], equal_var=False)
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: nan


### Colesterol 

In [1430]:
CON_PA.loc[:, 'colesterol total'] = pd.to_numeric(CON_PA['colesterol total'], errors='coerce')

promedio_col = CON_PA['colesterol total'].mean()
desviacion_estandar_col = CON_PA['colesterol total'].std()

resultado = f"{promedio_col:.2f} ± {desviacion_estandar_col:.2f}"
print("Mitjana ± Desviació Estàndard del colesterol total:", resultado)

Mitjana ± Desviació Estàndard del colesterol total: 132.10 ± 34.83


In [1431]:
CON_PA_MECVV.loc[:, 'colesterol total'] = pd.to_numeric(CON_PA_MECVV['colesterol total'], errors='coerce')

promedio_col = CON_PA_MECVV['colesterol total'].mean()
desviacion_estandar_col = CON_PA_MECVV['colesterol total'].std()

resultado = f"{promedio_col:.2f} ± {desviacion_estandar_col:.2f}"
print("Mitjana ± Desviació Estàndard del colesterol total:", resultado)

Mitjana ± Desviació Estàndard del colesterol total: 111.90 ± 28.37


In [1432]:
SIN_PA.loc[:, 'colesterol total'] = pd.to_numeric(SIN_PA['colesterol total'], errors='coerce')

promedio_col = SIN_PA['colesterol total'].mean()
desviacion_estandar_col = SIN_PA['colesterol total'].std()

resultado = f"{promedio_col:.2f} ± {desviacion_estandar_col:.2f}"
print("Mitjana ± Desviació Estàndard del colesterol total:", resultado)

Mitjana ± Desviació Estàndard del colesterol total: 145.27 ± 40.45


#### Pvalor

##### Normalitat 



In [1433]:
colconpa=CON_PA['colesterol total'].dropna()
stat, p_value = shapiro(colconpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9887
Valor p = 0.99048226
El número de files segueix una distribució normal.


In [1434]:
colconpamecvv=CON_PA_MECVV['colesterol total'].dropna()
stat, p_value = shapiro(colconpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9462
Valor p = 0.55290474
El número de files segueix una distribució normal.


In [1435]:
colsinpa=SIN_PA['colesterol total'].dropna()
stat, p_value = shapiro(colsinpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9725
Valor p = 0.84373822
El número de files segueix una distribució normal.


In [1436]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['colesterol total'], CON_PA_MECVV['colesterol total'], equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: nan


In [1437]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['colesterol total'], SIN_PA['colesterol total'], equal_var=False)
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: nan


In [1438]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['colesterol total'], SIN_PA['colesterol total'], equal_var=False)
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: nan


### Leucòcits

In [1439]:
CON_PA.loc[:, 'leucos'] = pd.to_numeric(CON_PA['leucos'], errors='coerce')

promedio_leu = CON_PA['leucos'].mean()
desviacion_estandar_leu = CON_PA['leucos'].std()

resultado = f"{promedio_leu:.2f} ± {desviacion_estandar_leu:.2f}"
print("Mitjana ± Desviació Estàndard dels leucòcits:", resultado)

Mitjana ± Desviació Estàndard de los leucocitos: 12.69 ± 4.88


In [1440]:
CON_PA_MECVV.loc[:, 'leucos'] = pd.to_numeric(CON_PA_MECVV['leucos'], errors='coerce')

promedio_leu = CON_PA_MECVV['leucos'].mean()
desviacion_estandar_leu = CON_PA_MECVV['leucos'].std()

resultado = f"{promedio_leu:.2f} ± {desviacion_estandar_leu:.2f}"
print("Mitjana ± Desviació Estàndard dels leucòcits:", resultado)

Mitjana ± Desviació Estàndard de los leucocitos: 13.08 ± 6.83


In [1441]:
SIN_PA.loc[:, 'leucos'] = pd.to_numeric(SIN_PA['leucos'], errors='coerce')

promedio_leu = SIN_PA['leucos'].mean()
desviacion_estandar_leu = SIN_PA['leucos'].std()

resultado = f"{promedio_leu:.2f} ± {desviacion_estandar_leu:.2f}"
print("Mitjana ± Desviació Estàndard dels leucòcits:", resultado)

Mitjana ± Desviació Estàndard de los leucocitos: 11.52 ± 6.52


#### Pvalor 

##### Normalitat

In [1442]:
leuconpa=CON_PA['leucos'].dropna()
stat, p_value = shapiro(leuconpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8482
Valor p = 0.09127190
El número de files segueix una distribució normal.


In [1443]:
leuconpamecvv=CON_PA_MECVV['leucos'].dropna()
stat, p_value = shapiro(leuconpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8939
Valor p = 0.40148390
El número de files segueix una distribució normal.


In [1444]:
leusinpa=SIN_PA['leucos'].dropna()
stat, p_value = shapiro(leusinpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8178
Valor p = 0.00096236
El número de files no segueix una distribució normal.


In [1445]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['leucos'], CON_PA_MECVV['leucos'], equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: nan


In [1446]:
t_stat_13, p_val_13 = mannwhitneyu(CON_PA['leucos'], SIN_PA['leucos'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: nan


In [1447]:
t_stat_23, p_val_23 = mannwhitneyu(CON_PA_MECVV['leucos'], SIN_PA['leucos'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: nan


### Limfòcits %

In [1448]:
CON_PA.loc[:, 'limfos'] = pd.to_numeric(CON_PA['limfos'], errors='coerce')

promedio_limf = CON_PA['limfos'].mean()
desviacion_estandar_limf = CON_PA['limfos'].std()

resultado = f"{promedio_limf:.2f} ± {desviacion_estandar_limf:.2f}"
print("Mitjana ± Desviació Estàndard dels limfòcits %:", resultado)

Mitjana ± Desviació Estàndard de los limfocitos %: 16.70 ± 12.29


In [1449]:
CON_PA_MECVV.loc[:, 'limfos'] = pd.to_numeric(CON_PA_MECVV['limfos'], errors='coerce')

promedio_limf = CON_PA_MECVV['limfos'].mean()
desviacion_estandar_limf = CON_PA_MECVV['limfos'].std()

resultado = f"{promedio_limf:.2f} ± {desviacion_estandar_limf:.2f}"
print("Mitjana ± Desviació Estàndard delss limfòcits %:", resultado)

Mitjana ± Desviació Estàndard de los limfocitos %: 17.20 ± 9.35


In [1450]:
SIN_PA.loc[:, 'limfos'] = pd.to_numeric(SIN_PA['limfos'], errors='coerce')

promedio_limf = SIN_PA['limfos'].mean()
desviacion_estandar_limf = SIN_PA['limfos'].std()

resultado = f"{promedio_limf:.2f} ± {desviacion_estandar_limf:.2f}"
print("Mitjana ± Desviació Estàndard dels limfòcits %:", resultado)

Mitjana ± Desviació Estàndard de los limfocitos %: 19.50 ± 11.57


#### Pvalor

##### Normalitat

In [1451]:
limfconpa=CON_PA['limfos'].dropna()
stat, p_value = shapiro(limfconpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8208
Valor p = 0.04757269
El número de files no segueix una distribució normal.


In [1452]:
limfconpamecvv=CON_PA_MECVV['limfos'].dropna()
stat, p_value = shapiro(limfconpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9374
Valor p = 0.51710593
El número de files segueix una distribució normal.


In [1453]:
limfsinpa=SIN_PA['limfos'].dropna()
stat, p_value = shapiro(limfsinpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9290
Valor p = 0.13154504
El número de files segueix una distribució normal.


In [1454]:
stat_12, p_val_12 = mannwhitneyu(CON_PA['limfos'], CON_PA_MECVV['limfos'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: nan


In [1455]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['limfos'], SIN_PA['limfos'], equal_var=False)
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: nan


In [1456]:
stat_23, p_val_23 = mannwhitneyu(CON_PA_MECVV['limfos'], SIN_PA['limfos'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: nan


### Proteïna C Reactiva

In [1457]:
CON_PA.loc[:, 'prot C react'] = pd.to_numeric(CON_PA['prot C react'], errors='coerce')

promedio_protcr = CON_PA['prot C react'].mean()
desviacion_estandar_protcr = CON_PA['prot C react'].std()

resultado = f"{promedio_protcr:.2f} ± {desviacion_estandar_protcr:.2f}"
print("Mitjana ± Desviació Estàndard de la proteïna C reactiva:", resultado)

Mitjana ± Desviació Estàndard de la proteina C reactiva: 8.90 ± 5.47


In [1458]:
CON_PA_MECVV.loc[:, 'prot C react'] = pd.to_numeric(CON_PA_MECVV['prot C react'], errors='coerce')

promedio_protcr = CON_PA_MECVV['prot C react'].mean()
desviacion_estandar_protcr = CON_PA_MECVV['prot C react'].std()

resultado = f"{promedio_protcr:.2f} ± {desviacion_estandar_protcr:.2f}"
print("Mitjana ± Desviació Estàndard de la proteïna C reactiva:", resultado)

Mitjana ± Desviació Estàndard de la proteina C reactiva: 8.88 ± 3.63


In [1459]:
SIN_PA.loc[:, 'prot C react'] = pd.to_numeric(SIN_PA['prot C react'], errors='coerce')

promedio_protcr = SIN_PA['prot C react'].mean()
desviacion_estandar_protcr = SIN_PA['prot C react'].std()

resultado = f"{promedio_protcr:.2f} ± {desviacion_estandar_protcr:.2f}"
print("Mitjana ± Desviació Estàndard de la proteïna C reactiva:", resultado)

Mitjana ± Desviació Estàndard de la proteina C reactiva: 6.16 ± 6.19


#### Pvalor

##### Normalitat

In [1460]:
pcrconpa=CON_PA['prot C react'].dropna()
stat, p_value = shapiro(pcrconpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9423
Valor p = 0.60665876
El número de files segueix una distribució normal.


In [1461]:
pcrconpamecvv=CON_PA_MECVV['prot C react'].dropna()
stat, p_value = shapiro(pcrconpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.7886
Valor p = 0.08325234
El número de files segueix una distribució normal.


In [1462]:
pcrsinpa=SIN_PA['prot C react'].dropna()
stat, p_value = shapiro(pcrsinpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8698
Valor p = 0.00636559
El número de files no segueix una distribució normal.


In [1463]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['prot C react'], CON_PA_MECVV['prot C react'], equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: nan


In [1464]:
t_stat_13, p_val_13 = mannwhitneyu(CON_PA['prot C react'], SIN_PA['prot C react'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: nan


In [1465]:
t_stat_23, p_val_23 = mannwhitneyu(CON_PA_MECVV['prot C react'], SIN_PA['prot C react'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: nan


### Urea 

In [1466]:
CON_PA.loc[:, 'urea'] = pd.to_numeric(CON_PA['urea'], errors='coerce')

promedio_urea = CON_PA['urea'].mean()
desviacion_estandar_urea = CON_PA['urea'].std()

resultado = f"{promedio_urea:.2f} ± {desviacion_estandar_urea:.2f}"
print("Mitjana ± Desviació Estàndard de la urea:", resultado)

Mitjana ± Desviació Estàndard de la urea: 62.17 ± 36.37


In [1467]:
CON_PA_MECVV.loc[:, 'urea'] = pd.to_numeric(CON_PA_MECVV['urea'], errors='coerce')

promedio_urea = CON_PA_MECVV['urea'].mean()
desviacion_estandar_urea = CON_PA_MECVV['urea'].std()

resultado = f"{promedio_urea:.2f} ± {desviacion_estandar_urea:.2f}"
print("Mitjana ± Desviació Estàndard de la urea:", resultado)

Mitjana ± Desviació Estàndard de la urea: 62.37 ± 37.72


In [1468]:
SIN_PA.loc[:, 'urea'] = pd.to_numeric(SIN_PA['urea'], errors='coerce')

promedio_urea = SIN_PA['urea'].mean()
desviacion_estandar_urea = SIN_PA['urea'].std()

resultado = f"{promedio_urea:.2f} ± {desviacion_estandar_urea:.2f}"
print("Mitjana ± Desviació Estàndard de la urea:", resultado)

Mitjana ± Desviació Estàndard de la urea: 55.03 ± 30.66


#### Pvalor

##### Normalitat

In [1469]:
uconpa=CON_PA['urea'].dropna()
stat, p_value = shapiro(uconpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8842
Valor p = 0.17374314
El número de files segueix una distribució normal.


In [1470]:
uconpamecvv=CON_PA_MECVV['urea'].dropna()
stat, p_value = shapiro(uconpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8991
Valor p = 0.42677381
El número de files segueix una distribució normal.


In [1471]:
usinpa=SIN_PA['urea'].dropna()
stat, p_value = shapiro(usinpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8289
Valor p = 0.00115364
El número de files no segueix una distribució normal.


In [1472]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['urea'], CON_PA_MECVV['urea'], equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: nan


In [1473]:
t_stat_13, p_val_13 = mannwhitneyu(CON_PA['urea'], SIN_PA['urea'], alternative='two-sided')
print("P-valor entre CON_PA i SIN_PA:", p_val_13)

P-valor entre CON_PA i SIN_PA: nan


In [1474]:
t_stat_23, p_val_23 = mannwhitneyu(CON_PA_MECVV['urea'], SIN_PA['urea'], alternative='two-sided')
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: nan


### Filtrat glomerular CDK-EPI

In [1475]:
CON_PA.loc[:, 'FGE CDK-EPI'] = pd.to_numeric(CON_PA['FGE CDK-EPI'], errors='coerce')

promedio_fg = CON_PA['FGE CDK-EPI'].mean()
desviacion_estandar_fg = CON_PA['FGE CDK-EPI'].std()

resultado = f"{promedio_fg:.2f} ± {desviacion_estandar_fg:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI:", resultado)

Mitjana ± Desviació Estàndard de la filtracion glomerular CDK-EPI: 56.00 ± 12.65


In [1476]:
CON_PA_MECVV.loc[:, 'FGE CDK-EPI'] = pd.to_numeric(CON_PA_MECVV['FGE CDK-EPI'], errors='coerce')

promedio_fg = CON_PA_MECVV['FGE CDK-EPI'].mean()
desviacion_estandar_fg = CON_PA_MECVV['FGE CDK-EPI'].std()

resultado = f"{promedio_fg:.2f} ± {desviacion_estandar_fg:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI:", resultado)

Mitjana ± Desviació Estàndard de la filtracion glomerular CDK-EPI: 54.67 ± 13.32


In [1477]:
SIN_PA.loc[:, 'FGE CDK-EPI'] = pd.to_numeric(SIN_PA['FGE CDK-EPI'], errors='coerce')

promedio_fg = SIN_PA['FGE CDK-EPI'].mean()
desviacion_estandar_fg = SIN_PA['FGE CDK-EPI'].std()

resultado = f"{promedio_fg:.2f} ± {desviacion_estandar_fg:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular CDK-EPI:", resultado)

Mitjana ± Desviació Estàndard de la filtracion glomerular CDK-EPI: 58.56 ± 18.40


#### Pvalor

##### Normalitat

In [1478]:
fgepiconpa=CON_PA['FGE CDK-EPI'].dropna()
stat, p_value = shapiro(fgepiconpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.7690
Valor p = 0.01994941
El número de files no segueix una distribució normal.


In [1479]:
fgepiconpamecvv=CON_PA_MECVV['FGE CDK-EPI'].dropna()
stat, p_value = shapiro(fgepiconpamecvv)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8120
Valor p = 0.14355397
El número de files segueix una distribució normal.


In [1480]:
fgepisinpa=SIN_PA['FGE CDK-EPI'].dropna()
stat, p_value = shapiro(fgepisinpa)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.9160
Valor p = 0.14543070
El número de files segueix una distribució normal.


In [1481]:
t_stat_12, p_val_12 = mannwhitneyu(CON_PA['FGE CDK-EPI'], CON_PA_MECVV['FGE CDK-EPI'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: nan


In [1482]:
t_stat_13, p_val_13 = mannwhitneyu(CON_PA['FGE CDK-EPI'], SIN_PA['FGE CDK-EPI'], alternative='two-sided')
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_13)

P-valor entre CON_PA i CON_PA_MECVV: nan


In [1483]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['FGE CDK-EPI'], SIN_PA['FGE CDK-EPI'], equal_var=False)
print("P-valor entre CON_PA_MECVV i SIN_PA:", p_val_23)

P-valor entre CON_PA_MECVV i SIN_PA: nan


### Filtrat glomerular MDRD

In [1484]:
CON_PA.loc[:, 'FGE MDRD'] = pd.to_numeric(CON_PA['FGE MDRD'], errors='coerce')

promedio_fgmd = CON_PA['FGE MDRD'].mean()
desviacion_estandar_fgmd = CON_PA['FGE MDRD'].std()

resultado = f"{promedio_fgmd:.2f} ± {desviacion_estandar_fgmd:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular MDRD:", resultado)

Mitjana ± Desviació Estàndard de la filtracion glomerular MDRD: 51.50 ± 0.71


In [1485]:
CON_PA_MECVV.loc[:, 'FGE MDRD'] = pd.to_numeric(CON_PA_MECVV['FGE MDRD'], errors='coerce')

promedio_fgmd = CON_PA_MECVV['FGE MDRD'].mean()
desviacion_estandar_fgmd = CON_PA_MECVV['FGE MDRD'].std()

resultado = f"{promedio_fgmd:.2f} ± {desviacion_estandar_fgmd:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular MDRD:", resultado)

Mitjana ± Desviació Estàndard de la filtracion glomerular MDRD: nan ± nan


In [1486]:
SIN_PA.loc[:, 'FGE MDRD'] = pd.to_numeric(SIN_PA['FGE MDRD'], errors='coerce')

promedio_fgmd = SIN_PA['FGE MDRD'].mean()
desviacion_estandar_fgmd = SIN_PA['FGE MDRD'].std()

resultado = f"{promedio_fgmd:.2f} ± {desviacion_estandar_fgmd:.2f}"
print("Mitjana ± Desviació Estàndard de la filtració glomerular MDRD:", resultado)

Mitjana ± Desviació Estàndard de la filtracion glomerular MDRD: 37.67 ± 17.43


#### Pvalor

##### Normalitat

In [1487]:
fgmd = CON_PA['FGE MDRD'].dropna()
stat, p_value = shapiro(fgmd)
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")


ValueError: Data must be at least length 3.

In [1488]:
stat, p_value = shapiro(CON_PA_MECVV['FGE MDRD'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

ValueError: Data must be at least length 3.

In [1489]:
stat, p_value = shapiro(SIN_PA['FGE MDRD'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.8f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal.")

Estadístic W = 0.8324
Valor p = 0.11252528
El número de files segueix una distribució normal.


In [1490]:
t_stat_12, p_val_12 = ttest_ind(CON_PA['FGE MDRD'], CON_PA_MECVV['FGE MDRD'], equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_12)

P-valor entre CON_PA i CON_PA_MECVV: nan


In [1491]:
t_stat_13, p_val_13 = ttest_ind(CON_PA['FGE MDRD'], SIN_PA['FGE MDRD'], equal_var=False)
print("P-valor entre CON_PA i CON_PA_MECVV:", p_val_13)

P-valor entre CON_PA i CON_PA_MECVV: nan


In [1492]:
t_stat_23, p_val_23 = ttest_ind(CON_PA_MECVV['FGE MDRD'], SIN_PA['FGE MDRD'], equal_var=False)
print("P-valor entre CON_PA_MECVV i CON_PA_MECVV:", p_val_23)

P-valor entre CON_PA_MECVV i CON_PA_MECVV: nan
